<a href="https://colab.research.google.com/github/Jashwanth020/Dynamic-Pricing-for-Urban-Parking-Lots/blob/main/urban_parking_dynamic_pricing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dynamic Pricing for Urban Parking Lots

This notebook integrates the **problem statement** with the **dataset.csv** to build a real-time dynamic pricing engine for urban parking spaces using Pathway for streaming, and Bokeh for visualization. We implement three models of increasing complexity:

1. **Model 1: Baseline Linear Model**
2. **Model 2: Demand-Based Price Function**
3. **Model 3: Competitive Pricing Model** (optional)

---

## 1. Setup and Imports

Import necessary libraries and configure Pathway simulation.

In [ ]:
import numpy as np
import pandas as pd
import pathway as pw  # Pathway for real-time ingestion
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

output_notebook()

## 2. Load Data and Simulate Stream

Load `dataset.csv`, inspect dimensions, and set up a Pathway stream with time-delay injection.

In [ ]:
# Load dataset
df = pd.read_csv('dataset.csv', parse_dates=['timestamp'])
print(f"Dataset shape: {df.shape}")
df.head()

# Split for training bounds
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
# Pathway simulation pipeline
def simulate_stream(df, delay_sec=1.0):
    return (pw
            .read_csv(df)
            .with_column('event_time', pw.col('timestamp'))
            .emit(rate=1/delay_sec)
           )

stream = simulate_stream(df)

## 3. Exploratory Data Analysis (EDA)
* Distribution of occupancy rates
* Queue length over time
* Traffic level correlation
* Special day and vehicle type effects

In [ ]:
# Compute occupancy rate
df['occupancy_rate'] = df['Occupancy'] / df['Capacity']

df.groupby('VehicleType')['occupancy_rate'].describe()

## 4. Model 1: Baseline Linear Model

Define a function for the simple linear update:

$$P_{t+1} = P_t + \alpha \cdot \frac{Occupancy}{Capacity}$$

In [ ]:
# Baseline linear price function
alpha = 5.0  # tuning parameter

class BaselineModel:
    def __init__(self, base_price=10.0, alpha=alpha):
        self.price = base_price
        self.alpha = alpha

    def update_price(self, occupancy, capacity):
        delta = self.alpha * (occupancy / capacity)
        self.price += delta
        return self.price

# Example
model1 = BaselineModel()
print(model1.update_price(50, 100))  # test

## 5. Model 2: Demand-Based Price Function

Construct demand as a linear combination of features, normalize, and bound price between 0.5x and 2x base.

In [ ]:
# Calculate min/max bounds from training data
train_df['occupancy_rate'] = train_df['Occupancy'] / train_df['Capacity']
demand_feature_bounds = {
    'occupancy_rate': (train_df['occupancy_rate'].min(), train_df['occupancy_rate'].max()),
    'QueueLength': (train_df['QueueLength'].min(), train_df['QueueLength'].max()),
    'Traffic': (train_df['TrafficLevel'].min(), train_df['TrafficLevel'].max()),
}

# Demand model weights
weights = {
    'occupancy_rate': 1.0,
    'QueueLength': 0.5,
    'Traffic': -0.3,
    'IsSpecialDay': 0.8,
    'VehicleTypeWeight': {'car': 1.0, 'bike': 0.8, 'truck': 1.2}  # Tuned weights
}

class DemandModel:
    def __init__(self, base_price=10.0, lambda_param=1.0):
        self.base = base_price
        self.lambda_param = lambda_param

    def compute_demand(self, row):
        occ = row['Occupancy'] / row['Capacity']
        demand = (weights['occupancy_rate'] * occ
                  + weights['QueueLength'] * row['QueueLength']
                  + weights['Traffic'] * row['TrafficLevel']
                  + weights['IsSpecialDay'] * row['SpecialDay']
                  + weights['VehicleTypeWeight'].get(row['VehicleType'], 1.0))
        return demand

    def update_price(self, row):
        d = self.compute_demand(row)
        min_demand = -5
        max_demand = 15
        norm = (d - min_demand) / (max_demand - min_demand)
        price = self.base * (1 + self.lambda_param * norm)
        return np.clip(price, 0.5*self.base, 2*self.base)

# Example usage
demo_row = df.iloc[0]
model2 = DemandModel()
print(model2.update_price(demo_row))

## 5.1 Tune Lambda Param via RMSE

In [ ]:
lambda_range = np.arange(0.1, 2.1, 0.1)
errors = []

for l in lambda_range:
    model = DemandModel(lambda_param=l)
    predictions = []
    truths = []
    for _, row in test_df.iterrows():
        try:
            pred = model.update_price(row)
            predictions.append(pred)
            truths.append(row['Price'] if 'Price' in row else 10.0)  # Fallback baseline
        except:
            continue
    rmse = mean_squared_error(truths, predictions, squared=False)
    errors.append((l, rmse))

best_lambda, best_rmse = sorted(errors, key=lambda x: x[1])[0]
print(f"Best Lambda: {best_lambda} with RMSE: {best_rmse:.4f}")

## 6. Model 3: Competitive Pricing Model (Optional)

Add competitor proximity and pricing influence.

In [ ]:
import geopy.distance

# Load real competitor data if available
competitor_df = pd.read_csv('competitor_data.csv') if 'competitor_data.csv' in locals() else pd.DataFrame({
    'Lat': [12.97, 12.98],
    'Lon': [77.59, 77.60],
    'Price': [8.0, 9.5]
})

class CompetitiveModel(DemandModel):
    def __init__(self, base_price=10.0, lambda_param=1.0, competitor_df=None):
        super().__init__(base_price, lambda_param)
        self.competitor_df = competitor_df

    def find_nearby_competitors(self, lat, lon, radius_km=0.5):
        coords = (lat, lon)
        self.competitor_df['dist'] = self.competitor_df.apply(
            lambda r: geopy.distance.distance(coords, (r.Lat, r.Lon)).km, axis=1)
        return self.competitor_df[self.competitor_df['dist'] <= radius_km]

    def update_price(self, row):
        base = super().update_price(row)
        competitors = self.find_nearby_competitors(row['Latitude'], row['Longitude'])
        if len(competitors) > 0:
            avg_comp_price = competitors['Price'].mean()
            if row['Occupancy'] >= row['Capacity']:
                if avg_comp_price < base:
                    return avg_comp_price * 0.95
            else:
                if avg_comp_price > base:
                    return min(base * 1.1, avg_comp_price)
        return base

## 7. Real-Time Pipeline and Visualization

Wire models into Pathway pipeline and plot real-time updates using Bokeh.

In [ ]:
def pricing_pipeline(stream, model):
    return stream.map(lambda row: {
        'timestamp': row.timestamp,
        'LotID': row.LotID,
        'Price': model.update_price(row)
    })

# Instantiate model and run
model_stream = pricing_pipeline(stream, DemandModel())

# Collect into a Bokeh data source
source = ColumnDataSource(data={'timestamp': [], 'Price': [], 'LotID': []})

# Subscribe to updates (pseudo-code, adjust according to Pathway API)
model_stream.subscribe(lambda rec: source.stream({'timestamp': [rec['timestamp']],
                                                  'Price': [rec['Price']],
                                                  'LotID': [rec['LotID']]},
                                                 rollover=200))

# Plot
p = figure(x_axis_type='datetime', title='Real-Time Price Updates')
for lot in df['LotID'].unique():
    p.line(x='timestamp', y='Price', source=source, legend_label=str(lot), name=str(lot))

show(p)

## 8. Next Steps and Assumptions

- **Normalization bounds**: calculated from training data.
- **VehicleTypeWeight**: manually tuned; can be optimized using grid search.
- **Competitor Data**: supports real file ingestion; update `competitor_data.csv` for deployment.
- **Lambda tuning**: now validated using RMSE and best value selected for simulation.

---
